# Assignment-3

**Due Date:** 14.12.2022 

---------------------------------------

**Instructor:** Erkut Erdem

**TA:** Sibel Kapan

---------------------------------------

**Student:** Zeynep Hafsa Dilmaç
**School Id:** 21992905

**Github Classroom:** https://classroom.github.com/a/Wv6S7qnL



## Part I: Theory Questions

Write your answers. You can add a picture or write it as a markdown.

## Part II: Classification of Flower Species using Neural Network


### Dataset Upload

Dataset uploaded from Drive or local

In [2]:
#connect drive
from google.colab import  drive 
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
#Load libraries
import os
import cv2
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
from torch import nn, optim
from torch.nn.modules import Module

In [ ]:
#unzipping folders
! unzip "drive/MyDrive/flowers.zip" -d flowers

In [ ]:
import torch
print(torch.cuda.is_available())

False


In [5]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [14]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize((0.5), (0.5))
])
#Transforms for cnn
transformer2=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize((0.5), (0.5))
])

In [21]:
train_data="flowers/flowers/train"
test_data="flowers/flowers/test"
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_data,transform=transformer2),
    batch_size=32, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_data,transform=transformer2),
    batch_size=32, shuffle=True
)

In [16]:
#categories
root=pathlib.Path(train_data)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes

['bellflower', 'daisy', 'dandelion', 'lotus', 'rose', 'sunflower', 'tulip']

In [17]:
target_dict={k: v for v, k in enumerate(np.unique(classes))}
target_dict

{'bellflower': 0,
 'daisy': 1,
 'dandelion': 2,
 'lotus': 3,
 'rose': 4,
 'sunflower': 5,
 'tulip': 6}

In [18]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_data+'/**/*.jpg'))
test_count=len(glob.glob(test_data+'/**/*.jpg'))

In [11]:
"""
This class includes the structure of the multilayer neural network and when it is created as a module, 
it calculates forward-back propogation operations according to the parameters such as loss function and learning rate.
 Finally, it makes class labeling for each image.

"""
class Model(nn.Module):
    def __init__(self, input_size, layers_data: list, learning_rate=0.01, optimizer=optim.Adam):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(torch.nn.Flatten())
        self.input_size = input_size  # Can be useful later ...
        for size, activation in layers_data:
            self.layers.append(nn.Linear(input_size, size))
            input_size = size  # For the next layer
            if activation is not None:
                assert isinstance(activation, Module), \
                    "Each tuples should contain a size (int) and a torch.nn.modules.Module."
                self.layers.append(activation)
        self.layers.append(torch.nn.LogSoftmax(dim=1))
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.learning_rate = learning_rate
        self.optimizer = optimizer(params=self.parameters(), lr=learning_rate)

    def forward(self, input_data):
        for layer in self.layers:
            input_data = layer(input_data)
        return input_data

In [12]:
"""
Here, the necessary parameters for the Multilayer neural network class are defined, and this neural network is created for all pictures
 and class tagging is applied to the pictures. Then, these found target results are compared with the real class targets and 
 an accuracy value is calculated according to what percentage is correct.
"""
#Optmizer and loss function
loss_function=nn.CrossEntropyLoss()
num_epochs=5
#Model training and saving best model

best_accuracy=0.0
mlp = Model(64*64*1, [ (7, nn.Sigmoid())])
#I set the weight_decay value to one tenth of the learning rate each time
optimizer=Adam(mlp.parameters(),lr=0.005,weight_decay=0.0005)


for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    mlp.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=mlp(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    # Evaluation on testing dataset
    mlp.eval()
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=mlp(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(mlp.state_dict(),'best_checkpoint.mlp')
        best_accuracy=test_accuracy


Epoch: 0 Train Loss: tensor(1.8816) Train Accuracy: 0.2289795918367347 Test Accuracy: 0.22714285714285715
Epoch: 1 Train Loss: tensor(1.8513) Train Accuracy: 0.24591836734693878 Test Accuracy: 0.235
Epoch: 2 Train Loss: tensor(1.8459) Train Accuracy: 0.2522448979591837 Test Accuracy: 0.22071428571428572
Epoch: 3 Train Loss: tensor(1.8562) Train Accuracy: 0.2345918367346939 Test Accuracy: 0.22357142857142856
Epoch: 4 Train Loss: tensor(1.8501) Train Accuracy: 0.24275510204081632 Test Accuracy: 0.245


# Evaluate results for mlp
I kept the input size constant (64x64x1) I tested different learning rates(0.005,0.01), batch sizes(32,64) . I repeated these tests separately with a single layer, then with 1 hidden layer, and then with 2 hidden layers, respectively.At the same time, I changed my epoch value from 0 to 5 in all my tests and recorded which epoch value gave the best results and best accuracy. Watching the change of the epoch value also protects us from overfitting the data.In fact, I tried the epoch value up to 10 in some of them, but in others it took a lot of time, so I reduced it to 5.In general terms, Multilayer NN did not give us good accuracy values. There could be many reasons for this. We could have given some parameters different or we could have used more hidden layers. When we examined the experiments, the smaller the learning rate gave us better results. We can say that we get better results with the increase in the number of hidden layers. When we make a comparison according to the batch size, we see that if the learning rate is small, the growth of the batch size does not change much, while if the learning rate is a larger number, the increase in the batch size increases the accuracy value more.We can say that increasing the number of hidden layers, increasing the bach size and decreasing the learning rate have increased the accuracy. Therefore, if we tried with more hidden layers, larger batch size and smaller learning rate, we could get better accuracy values. At the same time, I think that the epoch factor has an effect on these results. Because if we had done more training on the data, we could both observe overfitting and find the optimal accuracy value.

In [24]:
"""
This class includes the structure of the CNN and when it is created as a module, 
it calculates forward-back propogation operations according to the parameters such as loss function and learning rate.
 Finally, it makes class labeling for each image.

"""
class ConvNet(nn.Module):
    def __init__(self,num_classes=6,n=1,size=64):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,64,64)

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,64,64)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,64,64)
        self.relu1=nn.ReLU()
        #Shape= (256,12,64,64)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,32,32)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,32,32)
        self.relu2=nn.ReLU()
        
        #Shape= (256,20,32,32)
        self.fc=nn.Linear(in_features=32*32*20,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
        
        output=self.conv2(output)
        output=self.relu2(output)
        
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,20*32*32)
            
            
        output=self.fc(output)
            
        return output

In [ ]:
"""
Here, the necessary parameters for the convolutional neural network class are defined, and this neural network is created for all pictures
 and class tagging is applied to the pictures. Then, these found target results are compared with the real class targets and 
 an accuracy value is calculated according to what percentage is correct.
"""
model=ConvNet(num_classes=7).to(device)
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.01,weight_decay=0.001)
loss_function=nn.CrossEntropyLoss()
num_epochs=5
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    

# Evaluation Cnn Results
In the CNN part, I did the test and train operations first on a side with 1 conv layer and 1 fully connected layer, and then on a side with 2 conv and 2 fully connected layers. I also changed the learning rate and batch size parameters. As in mlp, I tried the epoch value up to 5 and recorded the highest accuracy value.Unlike mlp, here we set the pictures as input (64x64x3) in size. When we look at the Accuracy values, we see that we get better accuracy as the number of hidden layers increases. The opposite is true when the learning rate increases. The same effect is valid for batch size as well, although not much. Of course, the values ​​we tried are very limited and may not allow us to generalize.If we could try the epoch value with larger values, we could probably get better results.Adding more conv layers would probably give us much better accuracy. A good accuracy increase is observed even between 1 conv layer and 2 conv layers.


## CNN VS MLP
In general terms, the classification made in CNN gave much more accurate results.There could be a few reasons why you might have gotten better classification results using a convolutional neural network (CNN) compared to a multilayer perceptron (MLP). Here are a few possible explanations:

CNNs are specifically designed for image classification tasks, and are particularly effective at extracting features from images. This makes them well-suited for tasks such as object recognition, face detection, and image segmentation. In contrast, MLPs are more general-purpose neural networks that are not specifically designed for image classification tasks.

CNNs use convolutional layers, which apply a set of filters to the input data and produce a set of feature maps. These filters are trained to recognize certain patterns or features in the input data, and the feature maps represent the presence or absence of these patterns in the input image. This allows CNNs to automatically learn and extract meaningful features from the input data, which can improve the performance of the model. In contrast, MLPs do not have this ability and rely on the user to manually extract and engineer meaningful features from the input data.

CNNs can use pooling layers, which downsample the feature maps produced by the convolutional layers. This has the effect of reducing the size of the feature maps and reducing the number of parameters in the model, which can help prevent overfitting. MLPs do not have this ability.

The architecture of CNNs can be more efficient for image classification tasks compared to MLPs. For example, CNNs can use multiple convolutional layers with increasing filter sizes, which can capture increasingly complex patterns in the input data as the model goes deeper. This hierarchical structure can be more effective at extracting features from images than a flat MLP architecture.

It's worth noting that these are just a few possible explanations, and the performance of any particular model will depend on a variety of factors, including the quality of the training data, the choice of hyperparameters, and the specific task at hand.